In [2]:
import numpy as np
import pandas as pd
from scipy.linalg import svd

# 参考：PCA的数学原理

http://blog.codinglabs.org/articles/pca-tutorial.html

In [3]:
# 默认 一行 为一个sample。 每列为一个属性
X = np.array([[1, -1,1], [2, 1,2], [-3, 2,1], [1, 1,2], [2, 1,3], [3, 2,2]])
print(X.shape)
print(X)

XC = X - np.mean(X, axis=0,keepdims=True)
XC

(6, 3)
[[ 1 -1  1]
 [ 2  1  2]
 [-3  2  1]
 [ 1  1  2]
 [ 2  1  3]
 [ 3  2  2]]


array([[ 0.        , -2.        , -0.83333333],
       [ 1.        ,  0.        ,  0.16666667],
       [-4.        ,  1.        , -0.83333333],
       [ 0.        ,  0.        ,  0.16666667],
       [ 1.        ,  0.        ,  1.16666667],
       [ 2.        ,  1.        ,  0.16666667]])

# PCA 计算协方差矩阵时会减去均值(协方差的定义) 

In [8]:
import numpy as np
from sklearn.decomposition import PCA

# PCA 计算协方差矩阵时会减去均值(协方差的定义)。 
#代码见376-378行(减均值+SVD)： https://github.com/scikit-learn/scikit-learn/blob/412996f/sklearn/decomposition/pca.py#L105

# 默认一行是一个样本，列是特征。 PCA降为降的是行的维度
pca = PCA(n_components=2)
print(pca.fit_transform(X))

print('center:')
print(pca.fit_transform(XC))


[[-3.89363683e-03  2.15908343e+00]
 [-1.00652522e+00 -7.41792536e-02]
 [ 4.16259569e+00 -6.02573962e-01]
 [-3.84422851e-02 -5.15002105e-02]
 [-1.23717893e+00 -3.83180516e-01]
 [-1.87655562e+00 -1.04764948e+00]]
center:
[[-3.89363683e-03  2.15908343e+00]
 [-1.00652522e+00 -7.41792536e-02]
 [ 4.16259569e+00 -6.02573962e-01]
 [-3.84422851e-02 -5.15002105e-02]
 [-1.23717893e+00 -3.83180516e-01]
 [-1.87655562e+00 -1.04764948e+00]]


# X减去均值后进行SVD,结果才和PCA一样

In [7]:
# Truncte SVD
from sklearn.decomposition import TruncatedSVD
# tsvd 使用svd 来实现：https://github.com/scikit-learn/scikit-learn/blob/412996f/sklearn/decomposition/truncated_svd.py#L25


# 默认一行是一个样本，列是特征。 PCA降为降的是行的维度
tsvd = TruncatedSVD(2)
print(tsvd.fit_transform(X))
print('center:')
print(tsvd.fit_transform(XC))

[[ 1.02138507 -0.92248091]
 [ 2.99927574  0.06586869]
 [-0.74717548  3.66377642]
 [ 2.3180092   0.73472539]
 [ 3.65822947  0.44918799]
 [ 3.99937748  0.03395549]]
center:
[[-3.89363683e-03  2.15908343e+00]
 [-1.00652522e+00 -7.41792536e-02]
 [ 4.16259569e+00 -6.02573962e-01]
 [-3.84422851e-02 -5.15002105e-02]
 [-1.23717893e+00 -3.83180516e-01]
 [-1.87655562e+00 -1.04764948e+00]]


# TruncatedSVD 可由SVD 得到。 取前n个奇异值对应的奇异向量

注意：https://stats.stackexchange.com/questions/134282/relationship-between-svd-and-pca-how-to-use-svd-to-perform-pca
svd 的实现有符号的问题，可见下面的结果：http://amsword.is-programmer.com/posts/35966.html

http://www.kolda.net/publication/bracko08/

http://forums.fast.ai/t/svd-sign-ambiguity-for-pca-determinism/12480

具体实现：https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/extmath.py#L499


In [64]:

def sign_flip(u,v):
    max_abs_cols = np.argmax(np.abs(u), axis=0)
    #print(max_abs_cols)
    signs = np.sign(u[max_abs_cols, range(u.shape[1])])
    u *= signs
    v *= signs[:, np.newaxis]
    return u,v
u = np.array([[-8.05015,8.6175411, -1.6796562],
 [-2.0810077,  2.960713, -8.8886540],
 [ 8.60623642,  2.40505, -5.9729112],
 [-7.94800694,  2.05552,  1.42645168],
 [-2.55788819,  1.52938,  7.6698446],
 [-3.8798102,  4.18147, -5.9304835]])
v = np.array([[-0.96808293,  0.09805253, -0.23065371],
 [ 0.02267904,  0.95079119,  0.30900126],
 [-0.24960187, -0.29390784,  0.92266846]])
u,v = sign_flip(u,v)
print(u)
print(v)
np.argmax(np.abs(u), axis=0)

[[-8.05015     8.6175411   1.6796562 ]
 [-2.0810077   2.960713    8.888654  ]
 [ 8.60623642  2.40505     5.9729112 ]
 [-7.94800694  2.05552    -1.42645168]
 [-2.55788819  1.52938    -7.6698446 ]
 [-3.8798102   4.18147     5.9304835 ]]
[[-0.96808293  0.09805253 -0.23065371]
 [ 0.02267904  0.95079119  0.30900126]
 [ 0.24960187  0.29390784 -0.92266846]]


array([2, 0, 1])

In [67]:
# SVD
from scipy.linalg import svd
def hand_tsvd(X):
    U, S, V = svd(X, full_matrices=False)
    #U, S, V = svd(X)
    print(S)
    print(U.shape, S.shape, V.shape)
    
    # 处理符号问题, 保证U的行向量（V的列向量），每个向量中最大绝对值的量是正的。 （稳定的输出）
    # 需要以U或者V中一个作为基准，如将U作为基准，则若U的一行变了符号，那么对应V的一列符号也要变。
    # 参考：https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/extmath.py#L499

    U,V = sign_flip(U,V)
    print(U.shape, S.shape, V.shape)

    # 复原
    #print(np.dot(U,np.dot(np.diag(S), V)))
    
    # 转换1
    U2=U[:,:2]
    S2=np.diag(S[:2])
    
    print(np.dot(U2 , S2))
    
    # 转换2
    V1 = V.T
    V2 = V1[:,:2]
    print(np.dot(X, V2))

#hand_tsvd(X)
print('center:')
hand_tsvd(XC)

center:
[4.83672013 2.5054517  1.07804617]
(6, 3) (3,) (3, 3)
(6, 3) (3,) (3, 3)
[[-3.89363683e-03  2.15908343e+00]
 [-1.00652522e+00 -7.41792536e-02]
 [ 4.16259569e+00 -6.02573962e-01]
 [-3.84422851e-02 -5.15002105e-02]
 [-1.23717893e+00 -3.83180516e-01]
 [-1.87655562e+00 -1.04764948e+00]]
[[-3.89363683e-03  2.15908343e+00]
 [-1.00652522e+00 -7.41792536e-02]
 [ 4.16259569e+00 -6.02573962e-01]
 [-3.84422851e-02 -5.15002105e-02]
 [-1.23717893e+00 -3.83180516e-01]
 [-1.87655562e+00 -1.04764948e+00]]
